In [74]:
# 0) Imports
from pathlib import Path
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401


# Analyze dEchorate RIRs

This notebook loads the dEchorate metadata + HDF5 RIRs, filters a single room/source,
extracts early RIR segments per microphone, and prepares normalized tensors.


## Config
Point to your local CSV metadata and HDF5 file, and choose a room/source.
The metadata loader expects a comma-separated .csv.


In [75]:
# 1) User config (EDIT THESE)
METADATA_PATH = Path("../data/dEchorate/dEchorate_database.csv")  # CSV only
H5_PATH = Path("../data/dEchorate/dEchorate_rirs_gzip7.hdf5")


T0 = 4096  # number of samples from start of RIR (direct path + early echoes)

# mic-position normalization constants (approx room size used in dEchorate)
ROOM_DIMS = np.array([6.0, 6.0, 2.4], dtype=np.float32)

# If your IDs in the files are 1-based, set this to 1.
ID_BASE_OFFSET = 0  # typical is 0, but some datasets use 1


## Helpers
Metadata loading, column resolution, and inspection helpers.


## Load and inspect metadata
This cell loads metadata, prints columns, and resolves column names robustly.


In [76]:
# 2) Helpers: metadata loading + column resolution


def load_metadata(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Metadata file not found: {path.resolve()}")
    if path.suffix.lower() != ".csv":
        raise ValueError(
            f"Unsupported metadata file type: {path.suffix}. Expected .csv"
        )

    df = pd.read_csv(path, sep=None, engine="python")
    df.columns = [str(c).strip() for c in df.columns]

    drop_cols = [c for c in df.columns if c == "" or str(c).startswith("Unnamed")]
    if drop_cols:
        df = df.drop(columns=drop_cols)

    print(f"Loaded metadata: {path.name} | rows={len(df):,} cols={len(df.columns)}")
    return df


In [77]:
# Load metadata
metadata_df = load_metadata(METADATA_PATH)
print(
    "Metadata columns (first 30): "
    + ", ".join(map(str, list(metadata_df.columns)[:30]))
)
print("Preview:")
display(metadata_df.head(5))


Loaded metadata: dEchorate_database.csv | rows=10,912 cols=41
Metadata columns (first 30): filename, src_id, src_ch, src_type, src_signal, src_pos_x, src_pos_y, src_pos_z, room_code, room_rfl_floor, room_rfl_ceiling, room_rfl_west, room_rfl_south, room_rfl_east, room_rfl_north, room_fornitures, room_temperature, rec_silence_dB, rec_artifacts, mic_type, mic_id, mic_ch, mic_pos_x, mic_pos_y, mic_pos_z, array_id, array_bar_x, array_bar_y, array_bar_z, array_bar_pos_x
Preview:


,filename,src_id,src_ch,src_type,src_signal,src_pos_x,src_pos_y,src_pos_z,room_code,room_rfl_floor,...,array_bar_pos_z,array_bar_view_x,array_bar_view_y,array_bar_view_z,mic_view_x,mic_view_y,mic_view_z,src_view_x,src_view_y,src_view_z
0,2020-01-22__22-48-02,99.0,99.0,silence,silence,10000,0.0,1.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-22__22-48-02,99.0,99.0,silence,silence,10000,0.0,1.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-22__22-48-02,99.0,99.0,silence,silence,10000,0.0,1.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-22__22-48-02,99.0,99.0,silence,silence,10000,0.0,1.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-22__22-48-02,99.0,99.0,silence,silence,10000,0.0,1.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
# 3) Filter to ONE CASE: fixed room + fixed source + chirp
ROOM_CODE = [0,1,2] # pick a room_code that exists in metadata
SRC_ID = [0,1,2]  # pick a src_id that exists in metadata
SIGNAL_NAME = "chirp"  # use chirp/ESS-derived RIR measurements

subset_df = metadata_df.query(
    "room_code == @ROOM_CODE and src_id == @SRC_ID and " "src_signal == @SIGNAL_NAME"
).copy()

print(
    f"Filtered case -> room={ROOM_CODE}, src={SRC_ID}, sig={SIGNAL_NAME} | rows={len(subset_df)}"
)

needed_cols = [
    "room_code",  # choose/fix the room state
    "src_id",  # choose/fix the source
    "src_signal",  # keep only "chirp" rows (RIR/ESS measurement)
    "mic_id",  # mic identifier (use mic_ch instead if that's what matches HDF5)
    "mic_pos_x",  # mic design target (continuous X_out)
    "mic_pos_y",
    "mic_pos_z",
    "room_rfl_west",
    "room_rfl_east",
    "room_rfl_north",
    "room_rfl_south",
    "room_rfl_ceiling",
    "room_rfl_floor",
]

subset_df = subset_df[needed_cols]
display(subset_df)



Filtered case -> room=[0, 1, 2], src=[0, 1, 2], sig=chirp | rows=186


,room_code,src_id,src_signal,mic_id,mic_pos_x,mic_pos_y,mic_pos_z,room_rfl_west,room_rfl_east,room_rfl_north,room_rfl_south,room_rfl_ceiling,room_rfl_floor
5115,0.0,0.0,chirp,0.0,0.80316092,383.141.445,104.391.528,0.0,0.0,0.0,0.0,0.0,0.0
5116,0.0,0.0,chirp,1.0,0.8406819,384.527.719,104.391.528,0.0,0.0,0.0,0.0,0.0,0.0
5117,0.0,0.0,chirp,2.0,0.88758314,386.260.561,104.391.528,0.0,0.0,0.0,0.0,0.0,0.0
5118,0.0,0.0,chirp,3.0,0.94855474,388.513.257,104.391.528,0.0,0.0,0.0,0.0,0.0,0.0
5119,0.0,0.0,chirp,4.0,10.423.572,391.978.942,104.391.528,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9295,1.0,2.0,chirp,26.0,304.939.035,344.406.817,149.048.013,0.0,0.0,1.0,0.0,0.0,0.0
9296,1.0,2.0,chirp,27.0,300.846.949,34.727.994,149.048.013,0.0,0.0,1.0,0.0,0.0,0.0
9297,1.0,2.0,chirp,28.0,295.527.238,351.015,149.048.013,0.0,0.0,1.0,0.0,0.0,0.0
9298,1.0,2.0,chirp,29.0,287.343.067,356.761.246,149.048.013,0.0,0.0,1.0,0.0,0.0,0.0


## Explore HDF5 contents
List dataset candidates and infer axes.


In [79]:
# 4) HDF5: inspect keys, find RIR dataset, infer axis order
def list_h5_tree(h5obj, prefix=""):
    """Recursively list groups/datasets (short)."""
    for k in h5obj.keys():
        item = h5obj[k]
        if isinstance(item, h5py.Dataset):
            print(f"H5 dataset: {prefix}{k} shape={item.shape} dtype={item.dtype}")
        else:
            print(f"H5 group:   {prefix}{k}/")
            list_h5_tree(item, prefix=prefix + k + "/")


def find_candidate_rir_datasets(h5obj):
    """Return list of dataset paths that look like RIR tensors (>=3 dims)."""
    cands = []

    def _walk(obj, path=""):
        for k in obj.keys():
            item = obj[k]
            p = f"{path}/{k}" if path else k
            if isinstance(item, h5py.Dataset):
                name = k.lower()
                if ("rir" in name or "rirs" in name or "ir" == name) and item.ndim >= 3:
                    cands.append(p)
            else:
                _walk(item, p)

    _walk(h5obj)
    return cands


def infer_axes(shape, expected_mics=(30,), expected_srcs=(6,), expected_rooms=(11,)):
    """
    Heuristic inference: identify axes by dimension sizes.
    Returns dict with keys: time, mic, src, room
    """
    shape = list(shape)
    # time axis = largest dimension (RIR length)
    time_ax = int(np.argmax(shape))

    remaining = [i for i in range(len(shape)) if i != time_ax]

    # helper: pick axis whose size matches any of expected sizes
    def pick_axis(expected_sizes):
        for i in remaining:
            if shape[i] in expected_sizes:
                return i
        return None

    mic_ax = pick_axis(expected_mics)
    if mic_ax is not None:
        remaining.remove(mic_ax)

    src_ax = pick_axis(expected_srcs)
    if src_ax is not None:
        remaining.remove(src_ax)

    room_ax = pick_axis(expected_rooms)
    if room_ax is not None:
        remaining.remove(room_ax)

    return {"time": time_ax, "mic": mic_ax, "src": src_ax, "room": room_ax}


def extract_rir_segment(rirs_ds, axes, room_idx, src_idx, mic_idx, T0):
    """Slice a single RIR and return first T0 samples as 1D float32."""
    sl = [slice(None)] * rirs_ds.ndim
    if axes["room"] is not None:
        sl[axes["room"]] = int(room_idx)
    if axes["src"] is not None:
        sl[axes["src"]] = int(src_idx)
    if axes["mic"] is not None:
        sl[axes["mic"]] = int(mic_idx)

    x = rirs_ds[tuple(sl)]
    # Move time axis to front, flatten any leftovers
    x = np.moveaxis(x, axes["time"], 0)
    x = np.asarray(x).reshape(x.shape[0], -1)
    x = x[:, 0]  # pick first if extra singleton dims remain
    x = x[:T0].astype(np.float32, copy=False)
    return x


## Load HDF5 and extract early RIRs
Pick a dataset, infer axes, and build `Y_echo` for all mics in the subset.


In [80]:
if not H5_PATH.exists():
    raise FileNotFoundError(f"HDF5 file not found: {H5_PATH.resolve()}")

with h5py.File(H5_PATH, "r") as f:
    print(f"Opened HDF5: {H5_PATH.name}")
    # optional: uncomment if you want a full tree log
    # list_h5_tree(f)

    candidates = find_candidate_rir_datasets(f)
    if not candidates:
        # fallback: list top-level to help debugging
        print("No obvious RIR datasets found (by name). Top-level keys are:")
        print(list(f.keys()))
        raise KeyError(
            "Could not auto-detect RIR dataset. Inspect keys and set it manually."
        )

    print("Candidate RIR datasets:")
    for p in candidates:
        ds = f[p]
        print(f"  - {p} shape={ds.shape} dtype={ds.dtype}")

    # Choose the first candidate by default
    RIR_PATH = candidates[0]
    rirs_ds = f[RIR_PATH]
    print(f"Selected RIR dataset: {RIR_PATH} | shape={rirs_ds.shape}")

    axes = infer_axes(
        rirs_ds.shape, expected_mics=(30,), expected_srcs=(6,), expected_rooms=(11,)
    )
    print(f"Inferred axes: {axes} (None means 'not detected')")

    # If src/room axes weren't detected, we can still try slicing by assuming axis order,
    # but simplest is to print shape and set axes manually.
    if axes["mic"] is None or axes["src"] is None or axes["room"] is None:
        print("Could not infer all axes reliably.")
        print("Print rirs_ds.shape and set axes manually if needed.")
        print(f"rirs_ds.shape={rirs_ds.shape}")

    # 5) Extract Y_echo: early RIR segments for each mic
    room_idx = ROOM_CODE - ID_BASE_OFFSET
    src_idx = SRC_ID - ID_BASE_OFFSET

    Y_list = []
    for m in mic_indices:
        seg = extract_rir_segment(
            rirs_ds, axes, room_idx=room_idx, src_idx=src_idx, mic_idx=m, T0=T0
        )
        Y_list.append(seg)

    Y_echo = np.stack(Y_list, axis=0)  # (N, T0)
    print(f"Extracted Y_echo (raw) | shape={Y_echo.shape} dtype={Y_echo.dtype}")


Opened HDF5: dEchorate_rirs_gzip7.hdf5


RuntimeError: Unable to get group info (addr overflow, addr = 2536, size = 328, eoa = 2048)

## Inspect raw RIRs
Quick waveform plots and basic statistics.


In [ ]:
print("Plotting a few raw RIR segments...")
plt.figure(figsize=(10, 4))
for i in range(min(5, Y_echo.shape[0])):
    plt.plot(Y_echo[i], alpha=0.8)
plt.title("Raw RIR segments (first few mics)")
plt.xlabel("Sample")
plt.ylabel("Amplitude")
plt.tight_layout()
plt.show()

print("Y_echo summary:")
print(f"  shape: {Y_echo.shape}")
print(f"  min/max: {Y_echo.min():.4g} / {Y_echo.max():.4g}")
print(f"  mean/std: {Y_echo.mean():.4g} / {Y_echo.std():.4g}")


## Normalize Y and X
Normalize the RIR segments and the microphone design matrix.


In [ ]:
# 6) Normalization (simple, consistent)
# 6.1 Normalize Y_echo (waveform features)
# - remove per-sample DC offset per RIR segment
Y_echo = Y_echo - Y_echo.mean(axis=1, keepdims=True)

# - global standardization across the entire subset (recommended)
mu = float(Y_echo.mean())
sigma = float(Y_echo.std() + 1e-8)
Y_echo_norm = (Y_echo - mu) / sigma

# optional clipping for stability
Y_echo_norm = np.clip(Y_echo_norm, -5.0, 5.0).astype(np.float32)

print(f"Y_echo normalization: mu={mu:.6g}, sigma={sigma:.6g}")
print(
    "Y_echo_norm stats: mean=%.4f, std=%.4f" % (Y_echo_norm.mean(), Y_echo_norm.std())
)

# 6.2 Normalize X_design
if use_positions:
    X_design = X_design.astype(np.float32)
    # simple room-dimension scaling
    X_design_norm = X_design / ROOM_DIMS.reshape(1, 3)
    print(f"X_design_norm (positions/ROOM_DIMS) | shape={X_design_norm.shape}")
else:
    # discrete labels
    X_design_norm = X_design
    print("X_design is discrete mic indices; no normalization applied.")


## Inspect mic positions
Ranges and a 3D scatter when positions are available.


In [ ]:
if use_positions:
    print("Mic position ranges (meters):")
    print(f"  x: [{X_design[:,0].min():.3f}, {X_design[:,0].max():.3f}]")
    print(f"  y: [{X_design[:,1].min():.3f}, {X_design[:,1].max():.3f}]")
    print(f"  z: [{X_design[:,2].min():.3f}, {X_design[:,2].max():.3f}]")

    # simple 3D scatter (matplotlib default colors)
    fig = plt.figure(figsize=(6, 5))
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter(X_design[:, 0], X_design[:, 1], X_design[:, 2], s=30)
    ax.set_title("Mic positions (meters) for the selected room+source")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    plt.tight_layout()
    plt.show()
else:
    print("No mic positions available; using discrete mic indices.")


## Final tensors
Ready to train inverse models. Optionally save to disk.


In [ ]:
print("READY FOR TRAINING:")
print(f"  Inverse input  (Y_echo_norm): {Y_echo_norm.shape}")
print(f"  Inverse output (X_design_norm): {np.shape(X_design_norm)}")

# Optional: save for reuse
# np.save("Y_echo_norm.npy", Y_echo_norm)
# np.save("X_design_norm.npy", X_design_norm)
# print("Saved: Y_echo_norm.npy, X_design_norm.npy")
